In [174]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras import layers, Model
import keras

In [175]:
def training(len_input, embed_space_size, embed_action_size):
    '''define layer'''
    # shape = (資料筆數, other_input dimension)
    input_space = keras.Input(shape=len_input[0], name='Space_input')
    input_action = keras.Input(shape=len_input[0], name='Action_input')
    input_other = keras.Input(shape=len_input, name='Other_input')

    space_embedding = layers.Embedding(input_dim=embed_space_size, output_dim=4, mask_zero=True, name='Space_embedding')
    action_embeding = layers.Embedding(input_dim=embed_action_size, output_dim=4, mask_zero=True, name='Action_embedding')
    
    input_concat_embedding = layers.Concatenate(name='Input_merging', axis=2)
    layer_dense = layers.Dense(units=3, activation='softmax')

    '''define forward'''
    inputs = [input_space, input_action, input_other]
    embeded_space = space_embedding(input_space)
    embeded_action = action_embeding(input_action)

    embeded_input = input_concat_embedding([embeded_space, embeded_action, input_other])
    output_prob = layer_dense(embeded_input)

    model = keras.Model(inputs=inputs, outputs=output_prob, name='classifier')

    return model

In [176]:
# x = np.arange(20).reshape(2, 2, 5)
# print(x)

# y = np.arange(20, 32).reshape(2, 2, 3)
# print(y)

# tf.keras.layers.Concatenate(axis=2)([x, y])

In [177]:
model = training((100, 1), 10, 10)
model.summary()

Model: "classifier"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Space_input (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
Action_input (InputLayer)       [(None, 100)]        0                                            
__________________________________________________________________________________________________
Space_embedding (Embedding)     (None, 100, 4)       40          Space_input[0][0]                
__________________________________________________________________________________________________
Action_embedding (Embedding)    (None, 100, 4)       40          Action_input[0][0]               
_________________________________________________________________________________________

---

prepare data

In [178]:
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')

In [179]:
train_input = train_df[['Team', 'No.', 'Space', 'Action']]
train_label = train_df[['Erros', 'Score']]

In [186]:
train_space = train_input[['Space']]
train_action = train_input[['Action']]
train_others = train_input[['Team', 'No.']]

train_others = train_others.astype({'No.': 'str'})

In [187]:
train_space_dummy = pd.get_dummies(train_space)
train_action_dummy = pd.get_dummies(train_action)
train_others_dummy = pd.get_dummies(train_others)

In [188]:
train_label.loc[:, 'Nothing'] = 0.0
train_label.loc[(train_label['Erros'] == 0) & (train_label['Score'] == 0), 'Nothing'] = 1.0

c:\Users\YiChen\anaconda3\envs\SportsScience\lib\site-packages\pandas\core\indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
c:\Users\YiChen\anaconda3\envs\SportsScience\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\Users\YiChen\anaconda3\envs\SportsScience\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in 

In [189]:
print(train_others_dummy.shape)
print(train_space_dummy.shape)
print(train_action_dummy.shape)
print(train_label.shape)

(900, 24)
(900, 14)
(900, 8)
(900, 3)


In [193]:
train_action_dummy.shape[1]

8

In [ ]:
model = training(train_others_dummy.shape, train_space_dummy.shape[1], train_action_dummy.shape[1])